### Question 1. Running Elastic (1 point)

# Note: Throughout course i used google gemini so my results will be different

In [47]:
!curl localhost:9200

{
  "name" : "0cc6361a58b0",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "KjTukG7gQjiVjPTRytLvSA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [48]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Question 2. Indexing the data (1 point)

In [51]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '0cc6361a58b0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KjTukG7gQjiVjPTRytLvSA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [56]:
index_name = "course-questions"

if not es_client.indices.exists(index=index_name):
    print(f"Index '{index_name}' does not exist. Creating it...")
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "question": {"type": "text"},
                "text": {"type": "text"},
                "section": {"type": "text"},
                "course": {"type": "keyword"} #only course is keyword
            }
        }
    }
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Index '{index_name}' created successfully!")
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")

Index 'course-questions' already exists. Skipping creation.


In [57]:
if es_client.indices.exists(index=index_name):
    print(f"Index '{index_name}' exists.")
else:
    print(f"Index '{index_name}' does NOT exist.")

Index 'course-questions' exists.


In [59]:
index.fit(documents)

### Q3. Searching

In [81]:
query = 'How do execute a command on a Kubernetes pod'

In [98]:
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [99]:
response = es_client.search(index=index_name, body=search_query)

In [100]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'WhF4g5cBWB-XZKh0JAm3',
  '_score': 32.013527,
  '_source': {'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:',
   'section': 'Course Management Form for Homeworks',
   'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key',
   'course': 'data-engineering-zoomcamp'}}]

### Question 4. Filtering (1 point)

In [101]:
query = 'How do execute a command on a Kubernetes pod'

In [116]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [117]:
response = es_client.search(index=index_name, body=search_query)

In [118]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'HBF4g5cBWB-XZKh0Jwp3',
  '_score': 44.56891,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '0BGAg5cBWB-XZKh0Iw1M',
  '_score': 44.56891,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> 

### Question 5. Building a prompt (1 point)

In [130]:
query = 'How do execute a command on a Kubernetes pod'

In [131]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [132]:
def search(query):
    boost = {'question':4.0, 'text':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'machine-learning-zoomcamp'},
        boost_dict=boost,
        num_results = 10
        )
    return results

In [133]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [134]:
len(prompt_template)

206

In [135]:
def build_prompt(query, search_results):
    prompt_template = """
        You are a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT from the FAQ document
        Use only the facts from the CONTEXT while answering the QUESTION
        If the CONTEXT does not contain the answer, output NONE
        QUESTION: {question}
        CONTEXT: {context}
        
        """.strip()
    context =""
    for doc in search_results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n " 
    #print(context)
    prompt = prompt_template.format(question = query, context =context).strip()
    #print(query)
    return prompt

In [136]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [143]:
prompt

'You are a course teaching assistant. \n        Answer the QUESTION based on the CONTEXT from the FAQ document\n        Use only the facts from the CONTEXT while answering the QUESTION\n        If the CONTEXT does not contain the answer, output NONE\n        QUESTION: How do execute a command on a Kubernetes pod\n        CONTEXT: section: 10. Kubernetes and TensorFlow Serving\n question: Kubernetes-dashboard\n answer: Deploy and Access the Kubernetes Dashboard\nLuke\n\n section: 10. Kubernetes and TensorFlow Serving\n question: Why cpu vals for Kubernetes deployment.yaml look like “100m” and “500m”? What does "m" mean?\n answer: In Kubernetes resource specifications, such as CPU requests and limits, the "m" stands for milliCPU, which is a unit of computing power. It represents one thousandth of a CPU core.\ncpu: "100m" means the container is requesting 100 milliCPUs, which is equivalent to 0.1 CPU core.\ncpu: "500m" means the container has a CPU limit of 500 milliCPUs, which is equival

In [144]:
len(prompt)

7236

#### Q6 Count Tokens

In [155]:
from google import genai
import os
os.environ["GOOGLE_API_KEY"] = "API KEY HERE"

In [181]:
client = genai.Client()

response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents="How do execute a command on a Kubernetes pod?"
    
        )
#print(response)

In [182]:
response.usage_metadata.total_token_count

1956

In [183]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=1946, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=1946)], prompt_token_count=10, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=10)], thoughts_token_count=None, tool_use_prompt_token_count=None, tool_use_prompt_tokens_details=None, total_token_count=1956, traffic_type=None)